In [2]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from twisted.internet import reactor
import scrapy
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
import re
def isUserLink(href):
    m = re.match('/u/\d+/\w+', href)
    if(m is None):
        return False
    return True
def isStoryLink(href):
    m = re.match('/s/\d+/\w+', href)
    if(m is None):
        return False
    return True
def isReviewLink(href):
    m = re.match('/r/\d+', href)
    if(m is None):
        return False
    return True

In [14]:
import logging

class FanFicSpider(scrapy.Spider):
    name = "fanFic"
    start_urls = [
        'https://www.fanfiction.net/s/13090372/1/This-Labyrinth-of-Suffering'
        #'https://www.fanfiction.net/s/13059900/1/La-For%C3%AAt-des-%C3%82mes-Bris%C3%A9es',
        #'https://www.fanfiction.net/r/8636004/'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'fanficResult.json'                        # Used for pipeline 2
    }
    
    
        
    def parseReview(self, response):
        reviewOf = response.xpath('//th').xpath('.//@href').extract_first()
        #print(response.xpath('//table[@id="gui_table1i"]//td').extract())
        for review in response.xpath('//table[@id="gui_table1i"]//td'):
            reviewBody = (review.xpath('.//div/text()').extract_first())
            reviewer = (review.xpath('./text()').extract_first())
            if(reviewer is ' '):
                reviewer = (review.xpath('./a/@href').extract_first())
                yield response.follow(reviewer, callback=self.parseUserPage)
            yield{
                'pageType': 'review',
                'reviewOf': reviewOf,
                'reviewer': reviewer,
                'reviewBody': reviewBody
            }
        
    def parseUserPage(self, response):
        profile_top = response.xpath('//div[@id="content_wrapper_inner"]')
        #print(response.xpath('//div[@class="z-list mystories"]').xpath(".//@href"))
        cnt = 0
        for elem in response.xpath('//div[@class="z-list mystories"]').xpath(".//@href"):
            if(isStoryLink(elem.extract())):
                yield response.follow(elem.extract(), callback=self.parse)
            #print(elem.extract())
            if(isReviewLink(elem.extract())):
                yield response.follow(elem.extract(), callback=self.parseReview)
        favorites = []
        for elem in response.xpath('//div[@class="z-list favstories"]'):
            favStory = '';
            favAuthor = ''
            for link in elem.xpath('./a//@href').extract():
                if(isUserLink(link)):
                    favAuthor = link
                    yield response.follow(link, callback=self.parseUserPage)
                elif(isStoryLink(link)):
                    favStory = link
                    yield response.follow(link, callback=self.parse)
                elif(isReviewLink(link)):
                    yield response.follow(link, callback=self.parseReview)
            favorites.append({
                'favStory' : favStory,
                'favAuthor': favAuthor
            })
        #print(favorites)
        yield {
            'pageType': 'user',
            'name': profile_top.xpath('.//span/text()').extract_first(),
            'favorites': favorites
        }
    def parse(self, response):
        profile_top = response.xpath('//div[@id="profile_top"]')
        #print(profile_top.extract())
        #print(response.xpath('//div[@id="profile_top"]').xpath('.//@href').extract())
        #pre_story_links
        storyType = response.xpath('//div[@id="pre_story_links"]').xpath('.//a/@href').extract()[-1]
        author = ''
        #get author
        for link in profile_top.xpath('.//@href'):
            if(isUserLink(link.extract())):
                author = link.extract()
                yield response.follow(author, callback=self.parseUserPage)
        
        yield {
            'pageType': 'story',
            'author': author,
            'title': response.xpath('//title/text()').extract_first(),
            'storyType': storyType
        }
from crochet import setup
setup()
configure_logging({'LOG_FORMAT': '%(levelname)s: %(message)s'})
def run_spider():
    runner = CrawlerRunner()
    d = runner.crawl(FanFicSpider)

run_spider()

INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'fanficResult.json', 'LOG_LEVEL': 30}


INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'fanficResult.json', 'LOG_LEVEL': 30}
